In [43]:
import requests
import pandas as pd
import json
from dateutil import parser

In [3]:
API_KEY = "4be5daab06c0a1f934a27e5814580751-3352ea772f807c91fd9a5f998493a229"
ACCOUNT_ID = "101-003-29998453-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [4]:
session = requests.Session()

In [5]:
session.headers.update({
    "Authorization" : f"Bearer {API_KEY}",
    "Content-Type" : "application/json"
})

In [6]:
params = dict(
    count = 10,
    granularity = "H1"
)

In [7]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [8]:
response = session.get(url, params=None, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
data = response.json()

In [33]:
data = None

In [11]:
instruments_list = data['instruments']

In [12]:
len(instruments_list)

127

In [13]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [14]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [15]:
instruments_dict = {}
for i in instruments_list:
    key = (i['name'])
    instruments_dict[key] = {k : i[k] for k in key_i}

In [16]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.05'}

In [17]:
with open("../data/insturments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [75]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data
        
def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()    
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        
        for p in prices:
            if p in candle:  # Check if the price type exists in the candle
                for o in ohlc:
                    new_dict[f"{p}_{o}"] = float(candle[p][o])  # Access the price data safely
            else:
                # If the price type is missing, skip or set default values
                for o in ohlc:
                    new_dict[f"{p}_{o}"] = None  # Set to None or a default value

        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
    

In [76]:
code, data = fetch_candles('EUR_USD', count=10, granularity="H4")
candles_df = get_candles_df(data=data)

In [77]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-10-23 17:00:00+00:00,17009,1.07732,1.07866,1.07673,1.07822,1.07723,1.07859,1.07665,1.07815,1.07740,1.07874,1.07680,1.07829
1,2024-10-23 21:00:00+00:00,7091,1.07834,1.07900,1.07794,1.07888,1.07817,1.07892,1.07786,1.07881,1.07852,1.07908,1.07802,1.07895
2,2024-10-24 01:00:00+00:00,15541,1.07889,1.07936,1.07812,1.07932,1.07881,1.07929,1.07805,1.07924,1.07897,1.07944,1.07820,1.07939
3,2024-10-24 05:00:00+00:00,33187,1.07933,1.08082,1.07704,1.08032,1.07925,1.08075,1.07697,1.08025,1.07941,1.08091,1.07711,1.08040
4,2024-10-24 09:00:00+00:00,29422,1.08032,1.08054,1.07887,1.07964,1.08023,1.08046,1.07878,1.07956,1.08040,1.08061,1.07895,1.07973
5,2024-10-24 13:00:00+00:00,33369,1.07966,1.08114,1.07950,1.08108,1.07958,1.08106,1.07942,1.08101,1.07974,1.08122,1.07959,1.08116
6,2024-10-24 17:00:00+00:00,14603,1.08108,1.08299,1.08108,1.08279,1.08100,1.08292,1.08100,1.08271,1.08116,1.08306,1.08116,1.08287
7,2024-10-24 21:00:00+00:00,6387,1.08266,1.08289,1.08201,1.08217,1.08256,1.08281,1.08194,1.08210,1.08277,1.08302,1.08208,1.08224
8,2024-10-25 01:00:00+00:00,10650,1.08218,1.08276,1.08200,1.08219,1.08210,1.08268,1.08192,1.08212,1.08226,1.08284,1.08208,1.08226


In [78]:
candles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    9 non-null      datetime64[ns, tzutc()]
 1   volume  9 non-null      int64                  
 2   mid_o   9 non-null      float64                
 3   mid_h   9 non-null      float64                
 4   mid_l   9 non-null      float64                
 5   mid_c   9 non-null      float64                
 6   bid_o   9 non-null      float64                
 7   bid_h   9 non-null      float64                
 8   bid_l   9 non-null      float64                
 9   bid_c   9 non-null      float64                
 10  ask_o   9 non-null      float64                
 11  ask_h   9 non-null      float64                
 12  ask_l   9 non-null      float64                
 13  ask_c   9 non-null      float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), in